<h2>5 共享内存和常量内存</h2>
<h3>5.1 CUDA共享内存概述</h3>
<p>GPU两种类型内存：板载内存（全局内存） 片上内存（共享内存）</p>
<p>共享内存常用于 快内线程通信通道 用于全局内存可编程管理缓存 高速暂存存储器</p>
<h4>5.1.1 共享内存</h4>
<p>每个SM有一个共享内存 使同一线程块中线程能互相协作 便于重用 且可减少全局内存带宽</p>
<p>__shared__ 在核函数中声明->作用域为核函数</p>
<p>__shared__ 在核函数外声明->全局</p>
<p>动态声明： extern __shared__ int tile[];</p>
<p>kernel<<<grid, block, isize*sizeof(int)>>>(...)</p>
<h4>5.1.3 共享内存存储体和访问模式</h4>
<h5>5.1.3.1 内存存储体</h5>
<p>共享内存被分为的内存模型：线程束发布共享内存加载/存储，每个存储体只访问不多于一个内存地址->可由一个内存事务完成</p>
<h5>5.1.3.2 存储体冲突</h5>
<p>共享内存中多个地址请求落在相同内存存储体中</p>
<p>并行访问：多地址->多存储体 串行访问：多->一 广播访问：一->一</p>
<h5>5.1.3.3 访问模式</h5>
<p>共享内存存储体宽度规定地址和存储体对应关系</p>
<h5>5.1.3.4 内存填充</h5>
<p>避免存储体冲突的一种方法 在每N（存储体数量）个元素之后添加一个字，改变从字到存储体映射</p>
<h5>5.1.3.5 访问模式设置</h5>
<p>cudaDeviceGetSharedMemConfig</p>
<h4>5.1.4 配置共享内存量</h4>
<p>共享内存/一级缓存共享SM的片上内存</p>
<p>CUDA两种方法配置：按设备 按核函数</p>
<p>cudaDeviceSetCacheConfig(cudaFuncCache cacheConfig)</p>
<p>核函数用到较多共享内存->SharedMem 较多寄存器->l1 cache</p>
<h4>5.1.5 同步</h4>
<p>同步的两个基本方法：障碍 内存栅栏</p>
<h5>5.1.5.1 弱排序内存模型</h5>
<p>GPU线程从不同内存中读取数据顺序和读指令在程序中出现顺序不一定相同</p>
<h5>5.1.5.2 显式障碍</h5>
<p>只在同一线程块线程间执行 void __syncthreads();
块中线程必须等待直到所有线程到达该点 被这些线程访问的所有全局和共享内存对同一块中所有线程可见</p>
<h5>5.1.5.3 内存栅栏</h5>
<p>确保栅栏前任何内存写操作对栅栏后其他线程可见 3种栅栏：块/网格/系统</p>
<p>void __threadfence_block(); 线程块 void __threadfence(); 线程 void __threadfence_system()全局内存 全部设备</p>

<h3>5.2 共享内存的数据布局</h3>
<h4>5.2.1 方形共享内存</h4>
<p>__shared__ int tile[N][N]</p>
<p>tile[threadIdx.y][threadIdx.x] 更优</p>
<h5>5.2.1.1 行主序访问和列主序访问</h5>
<h5>5.2.1.2 按行主序和按列主序读</h5>
<h5>5.2.1.3 动态共享内存</h5>
<h4>5.2.2 矩形共享内存</h4>
<p>__shared__ int tile[Row][Col]</p>
<h3>5.3 减少全局内存访问</h3>
<h4>5.3.1 使用共享内存的并行归约...</h4>
<h4>5.3.4 有效带宽</h4>
<p>有效带宽=（读字节+写字节）/（运行时间*10^9）GB/s</p>
<h3>5.4 合并的全局内存访问</h2>
<h4>5.4.1 基准转置内核</h3>
<h3>5.5 常量内存</h2>
<p>用于只读数据和统一访问线程束中线程的数据 对内核代码只读，对主机既可读又可写，位于设备DRAM上（和全局内存一样），有一个专用的片上缓存（64KB）</p>
<p>如果线程束中线程访问相同的位置，访问最优。访问不同地址，需要串行</p>
<p>__constant__</p>
<p>模板计算在所有线程系数相同且不会被修改，使其成为最优的候选->广播式访问模式</p>
<h4>5.5.2 与只读缓存的比较</h2>
<p>只读缓存在分散读取方面比一级缓存更好，当线程束线程都读取相同地址时，不应使用只读缓存</p>
<p>__ldg 或 全局内存限定指针</p>
<h3>5.6 线程束洗牌命令（shuffle instruction）</h3>
<p>只要两个线程在相同的线程束中，就允许这两个线程直接读取另一个线程寄存器</p>
<p>使线程束中线程彼此间可以直接交换数据，而不通过共享内存或全局内存</p>
<p>束内线程（lane）：线程束内的单一线程</p>
<p>laneID = threadIdx.x%32</p>
<p>warpID = threadIdx.x/32</p>
<p>一维线程块中线程洗牌ID可按以下公式：shuffleID = threadIdx.x % width</p>
<p>int y = __shfl(x,3,16); 线程0-15从线程3接收x值，线程16-31从线程19接收x的值</p>
<p>__shfl_up->右移x单位 最低的不变 __shfl_down左移同理 __shfl_xor亦或</p>
